# model 3

In [1]:
import polars as pl
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# import from the auxFunctions.py file
from auxFunctions import calculate_mae_and_mrrmse, mean_rowwise_rmse_loss, custom_mean_rowwise_rmse, create_model_checkpoint, plot_training_history

2023-11-13 05:15:42.884291: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 05:15:43.525192: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## pre-process data
one-hot encode cell types

In [2]:
de_train = pl.scan_parquet('./de_train.parquet')
de_train_df = de_train.collect().to_pandas()

# test provided by kaggle --> upload predictions to kaggle to get the score
id_map = pd.read_csv('./id_map.csv')

affinities = pd.read_csv('affinities_Transformer_CNN_BindingDB.csv', index_col=0)

train dataset provided by kaggle
- will be split into train/test/validation for internal testing before model is trained on the entire train and used to predict on the test in id_map

In [3]:
def extractAffinities(sm_names, affinities):
    """
    Function to extract affinities from the affinities dataframe

    Parameters:
    - sm_names: List/Array of sm_names
    - affinities: Stored affinities predicted using DeepPurpose

    Returns:
    - Affinities as a numpy array
    """
    encoded_affinities = []
    for name in sm_names:
        filtered = affinities[affinities['sm_name'] == name]
        sm_affinities = filtered.iloc[:, 2:].values[0]
        encoded_affinities.append(sm_affinities)

    np_encoded_affinities = np.array(encoded_affinities)

    return np_encoded_affinities

In [4]:
# one-hot encode cell_type
cell_type = de_train_df['cell_type'].to_numpy().reshape(-1, 1)
encoder = OneHotEncoder()
encoder.fit(cell_type)

# of type scipy.sparse._csr.csr_matrix
encoded_cell_type = encoder.transform(cell_type)

# map sm_name to affinities
sm_name = de_train_df['sm_name']

# has shape (614, 12766), of type numpy.ndarray
#this is the training set
np_encoded_affinities = extractAffinities(sm_name, affinities)

# concatenate encoded_cell_type and np_encoded_affinities
# final shape (614, 12772)
encoded_features = np.hstack((encoded_cell_type.toarray(), np_encoded_affinities))

print(encoded_cell_type)


# wanted output
genes_lfc = de_train_df.drop(columns=['cell_type', 'sm_name', 'sm_lincs_id', 'SMILES', 'control'])

  (0, 2)	1.0
  (1, 3)	1.0
  (2, 4)	1.0
  (3, 5)	1.0
  (4, 2)	1.0
  (5, 3)	1.0
  (6, 4)	1.0
  (7, 5)	1.0
  (8, 0)	1.0
  (9, 1)	1.0
  (10, 2)	1.0
  (11, 3)	1.0
  (12, 4)	1.0
  (13, 5)	1.0
  (14, 2)	1.0
  (15, 3)	1.0
  (16, 4)	1.0
  (17, 5)	1.0
  (18, 2)	1.0
  (19, 3)	1.0
  (20, 4)	1.0
  (21, 5)	1.0
  (22, 2)	1.0
  (23, 3)	1.0
  (24, 4)	1.0
  :	:
  (589, 5)	1.0
  (590, 2)	1.0
  (591, 3)	1.0
  (592, 4)	1.0
  (593, 5)	1.0
  (594, 2)	1.0
  (595, 3)	1.0
  (596, 4)	1.0
  (597, 5)	1.0
  (598, 2)	1.0
  (599, 3)	1.0
  (600, 4)	1.0
  (601, 5)	1.0
  (602, 2)	1.0
  (603, 3)	1.0
  (604, 4)	1.0
  (605, 5)	1.0
  (606, 2)	1.0
  (607, 3)	1.0
  (608, 4)	1.0
  (609, 5)	1.0
  (610, 2)	1.0
  (611, 3)	1.0
  (612, 4)	1.0
  (613, 5)	1.0


In [5]:
# repeat for kaggle test set
kaggle_cell_type = id_map['cell_type'].to_numpy().reshape(-1, 1)


encoded_kaggle_cell_type = encoder.transform(kaggle_cell_type)



kaggle_sm_name = id_map['sm_name']

#This is for the test data set
encoded_kaggle_affinities = extractAffinities(kaggle_sm_name, affinities)


# final shape (255, 12772)
encoded_kaggle_features = np.hstack((encoded_kaggle_cell_type.toarray(), encoded_kaggle_affinities))

In [6]:
# Split the data into 70% training, 15% validation, and 15% testing
x_train, x_temp, y_train, y_temp = train_test_split(encoded_features, genes_lfc.values, test_size=0.3, shuffle=False)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, shuffle=False)

# used for final training before predicting on kaggle_test
full_features = encoded_features
full_labels = genes_lfc.values

print(y_train)

[[ 0.10472047 -0.07752421 -1.62559604 ...  0.03412678  0.22137655
   0.36875538]
 [ 0.91595324 -0.88438038  0.37183448 ...  0.70477983  1.09670189
  -0.86988664]
 [-0.38772076 -0.30537826  0.56777737 ...  0.41576793  0.07843919
  -0.25936541]
 ...
 [ 0.61950832 -0.03779631  0.87478376 ... -0.95027954 -0.49932213
   0.11094977]
 [-0.1314054   0.17761662 -0.11689098 ...  0.25845771 -0.29531843
  -0.3699244 ]
 [ 1.07188372 -0.35765163  0.17995645 ... -0.48290495 -0.4353545
  -0.23529439]]


## model tuning

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

from keras import Sequential
from keras.layers import Activation, Dense

In [8]:
##define search space

def build_model(hp):
    model = keras.Sequential()

    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=18211, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )

    model.add(layers.Dense(18211, activation="linear"))


    model.compile(
        optimizer="adam", loss=mean_rowwise_rmse_loss, metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )

    return model

In [9]:
import keras_tuner

In [10]:
build_model(keras_tuner.HyperParameters())

2023-11-13 05:16:03.514551: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [11]:
#select tuner classs to run search

tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective=keras_tuner.Objective("val_root_mean_squared_error", direction="min"),
    max_trials=10,
    executions_per_trial=3,
    overwrite=True,
    directory="models",
    project_name="model_3",
)

In [12]:
#search space summary

tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 18211, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}


In [ ]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 7 Complete [00h 00m 36s]
val_root_mean_squared_error: 1.0060972770055134

Best val_root_mean_squared_error So Far: 0.9800203045209249
Total elapsed time: 00h 10m 17s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
4                 |5                 |num_layers
10720             |2400              |units_0
relu              |tanh              |activation
17472             |32                |units_1
1920              |32                |units_2
15520             |32                |units_3
5856              |32                |units_4

Epoch 1/2
14/14 [==============================] - 61s 4s/step - loss: 12.7084 - root_mean_squared_error: 33.0734 - val_loss: 0.8456 - val_root_mean_squared_error: 1.0070
Epoch 2/2
14/14 [==============================] - 66s 5s/step - loss: 1.3668 - root_mean_squared_error: 2.6369 - val_loss: 0.8372 - val_root_mean_squared_error: 0.9971
Epoch 1/2
14/14 [==============================] - 30s 2s/step - loss: 12.1871 

In [ ]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(614, 12772))
best_model.summary()

In [ ]:
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model_tuned = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, y_val))
y_all = np.concatenate((y_train, y_val))
model_tuned.fit(x=x_all, y=y_all, epochs=50)

In [ ]:
model.saved("model_3")
model.save_weights("model_3_weights.h5")

In [ ]:
calculate_mae_and_mrrmse(model=model_tuned, data=X_test, y_true=y_test)